In [2]:
# plotting stuff
# import matplotlib # for exporting graphs
# matplotlib.use('Agg') # for exporting graphs
# fig_size = (1.5*6.4, 1.5*4.8)
# font_size = 14

In [1]:
import time
import numpy
import matplotlib.pyplot as pyplot
import matplotlib.animation as animation
pyplot.rcParams['animation.html'] = 'jshtml'
import UtilitiesTDSEv2 as use
from tensorflow import keras

<h3>Numerical environment</h3>

In [2]:
# define time grid
num_time_steps = 500
initial_time = 0 # time units
final_time = 5 # time units

time_steps, time_resolution = numpy.linspace(initial_time,
                                             final_time,
                                             num_time_steps,
                                             retstep=True)


# define space grid
num_space_steps = 1024
left_space_bound = -25.6 # space units
right_space_bound = 25.6 # space units

space_steps, space_resolution = numpy.linspace(left_space_bound,
                                               right_space_bound,
                                               num_space_steps,
                                               retstep=True)
    

<h3>Training data generator</h3>

In [3]:
def training_data_generator(num_psi0,
                            num_potentials,
                            verbose=True):
    
    if verbose:
        print(f'Building training data, {num_psi0} samples:')
        # store start time
        start_time = time.time() 
        # initialize mid_time
        mid_time = start_time
        # initialize current_time
        current_time = start_time
    
    
    ###############################################################################
    # Random initial wavefunctions

    if verbose:
        print('>>> Generating random initial wavefunctions')
        mid_time = time.time()
    
    psi0_array = use.random_psi0(num_psi0, space_steps)
    
    if verbose:
        current_time = time.time()
        print(f'>>> Success! {current_time - mid_time}')
        mid_time = current_time
    
    ###############################################################################
    # Random potential functions
    
    if verbose:
        print('>>> Generating random potential')
        mid_time = time.time()
    
    # initialize array of potentials
    potential_array = []
    
    potential_array = use.random_potentials(num_potentials, space_steps)
    
    if verbose:
        current_time = time.time()
        print(f'>>> Success! {len(potential_array)} potential generated: {current_time - mid_time} sec')
        mid_time = current_time
    
    ###############################################################################
    # propagate random initial wavefunctions using pseudospectral method
    
    # initialize solution array
    time_series_array = []
    
    if verbose:
        print('>>> Solving pseudo-spectral ODE...')
        ode_solve_start_time = time.time()
        mid_time = ode_solve_start_time
    
    for p in range(num_potentials):
        for s in range(num_psi0):
            if verbose:
                # Give progress message every 5 potentials and 10 samples solved
                if p%5 == 0 and s%10 == 0: #and i != 0:
                    current_time = time.time()
                    print(f'    potential {p} of {num_potentials}, Psi0 {s} of {num_psi0},',
                          f'Batch time: {current_time - mid_time} sec')
                    # update value in mid_time with current time
                    mid_time = current_time
            
            # propagate s initial wavefunction and p potential
            time_series_array.append(use.sudo_spec_propagate(psi0_array[s],
                                                             potential_array[p],
                                                             time_steps,
                                                             space_steps))
    
    # change to numpy array so can use numpy functions (like delete)
    time_series_array = numpy.array(time_series_array)
    
    ###############################################################################
    # total time to solve all ODEs
    if verbose:
        current_time = time.time()
        print(f'>>> Success! Total time to solve all ODEs {current_time - ode_solve_start_time} sec')
        # total time
        print(f'Total elapse time: {current_time - start_time} sec')
    
    return time_series_array


In [24]:
num_potentials=5
num_psi0=5

raw_data = training_data_generator(num_psi0,
                                   num_potentials)

Building training data, 5 samples:
>>> Generating random initial wavefunctions
>>> Success! 0.024153709411621094
>>> Generating random potential
>>> Success! 5 potential generated: 0.006844997406005859 sec
>>> Solving pseudo-spectral ODE...
    potential 0 of 5, Psi0 0 of 5, Batch time: 1.1920928955078125e-06 sec
>>> Success! Total time to solve all ODEs 60.208019971847534 sec
Total elapse time: 60.23918080329895 sec


In [25]:
raw_data.shape # its grouped by potential function

(25, 2, 500, 1024)

<h3>Packager</h3>

The training data has the form:

(sample, input/output, time, space)

Samples are grouped by potential, so, if you have two potentials and three psi0, then the training data would be like:

(V0,psi0-0), (V0,psi0-1), (V0,psi0-2), (V1,psi0-0), (V1,psi0-1), (V1,psi0-2)

With the index:

(0, 2, 500, 1024), (1, 2, 500, 1024), (2, 2, 500, 1024), (3, 2, 500, 1024), (4, 2, 500, 1024), (5, 2, 500, 1024)

In [26]:
def training_data_packager(data,
                           test_set_rnd_seed = None,
                           test_set_split = 0.2, # percent
                           cat_complex_psi=True, # bool
                           verbose=True):
    
    if verbose:
        start_time = time.time()
        print('>>> Packaging training data....')
        

    # reshape so potential is a separate index
    # (num_potentials, num_psi0, input/output, num_time_steps, num_space_steps)
    data = data.reshape((num_potentials, num_psi0, 2, num_time_steps, num_space_steps))
    
    num_test_potentials = int(num_potentials*test_set_split)
    num_train_potentials = num_potentials - num_test_potentials
    
    # initialize random bit generator
    rnd_gen = numpy.random.default_rng(test_set_rnd_seed)

    # randomly select potentials for test set
    test_set_index_array = rnd_gen.integers(0, num_potentials, num_test_potentials)

    train_input = numpy.delete(data, test_set_index_array, axis=0)[:,:,0]
    if cat_complex_psi:
        train_input = numpy.concatenate([train_input.real, train_input.imag], axis=3)
        train_input = train_input.reshape((num_train_potentials*num_psi0, num_time_steps, 2*num_space_steps))
    else:
        train_input = train_input.reshape((num_train_potentials*num_psi0, num_time_steps, num_space_steps))

    train_output = numpy.delete(data, test_set_index_array, axis=0)[:,:,1]
    train_output = train_output.reshape((num_train_potentials*num_psi0, num_time_steps, num_space_steps))

    test_input = data[test_set_index_array,:,0]
    if cat_complex_psi:
        test_input = numpy.concatenate([test_input.real, test_input.imag], axis=3)
        test_input = test_input.reshape((num_test_potentials*num_psi0, num_time_steps, 2*num_space_steps))
    else:
        test_input = test_input.reshape((num_test_potentials*num_psi0, num_time_steps, num_space_steps))

    test_output = data[test_set_index_array,:,1]
    test_output = test_output.reshape((num_test_potentials*num_psi0, num_time_steps, num_space_steps))

    
    
    if verbose:
        print(f'>>> Success! {time.time() - start_time} sec')

    
    return ({'input': train_input, 'output': train_output},
                {'input': test_input, 'output': test_output})

In [27]:
packaged_train_data, packaged_test_data = training_data_packager(raw_data)
print(packaged_test_data['output'].shape)

>>> Packaging training data....
>>> Success! 0.1676177978515625 sec
(5, 500, 1024)


<h3>Setup and train neural network</h3>

In [34]:
num_epochs = 50

# WHAT IS THE FORM OF THE INPUT? IS IT COMPLEX?
# IF YOU ARE USING COMPLEX VALUES, YOU NEED TO CONSIDER THINGS LIKE THE ACTIVATION FUNCTION
# AND HOW IT HANDLES THINGS (FOR EXAMPLE GREATER/LESS THAN COMPARISON AREN'T DEFINED FOR COMPLEX NUMBERS)
# IF IT IS COMPLEX, HOW IS THAT BEING HANDLED? WHAT IS GOING ON WITH YOUR METHOD UNDER THE HOOD?
# NEED TO UNDERSTAND HOW THINGS ARE BEING HANDLED BY KERAS.

# IF IT IS COMPLEX NUMBERS BEING FED INTO THE NETWORK, YOU ARE GETTING A REAL NUMBER BACK, OR IT'S GIVING
# YOU A COMPLEX NUMBER BUT ALL OF YOUR TRAINING DATA IS REAL


# create input nodes
nn_input = keras.Input(shape=(500, 2048))

# create hidden layers, connect graph nodes (construct graph structure)
nn_output = keras.layers.Dense(units=2048, activation='elu')(nn_input)
nn_output = keras.layers.Dense(units=2048, activation='elu')(nn_output)
nn_output = keras.layers.Dense(1024)(nn_output)

# create a model from the graph
nn_model = keras.Model(inputs=nn_input, outputs=nn_output)

# complile model parameters
opt = keras.optimizers.Adam(learning_rate=0.0005)
nn_model.compile(optimizer=opt, loss='MSE')

# fit the model
nn_hist = nn_model.fit(x=packaged_train_data['input'],
                       y=packaged_train_data['output'],
                       batch_size=10,
                       validation_split=0.2,
                       epochs=num_epochs,
                       verbose=1
                      )

Epoch 1/50
2/2 [==============================] - 3s 1s/step - loss: 0.0220 - val_loss: 0.0094
Epoch 2/50
2/2 [==============================] - 3s 981ms/step - loss: 0.0075 - val_loss: 0.0077
Epoch 3/50
2/2 [==============================] - 3s 1s/step - loss: 0.0061 - val_loss: 0.0071
Epoch 4/50
2/2 [==============================] - 2s 976ms/step - loss: 0.0050 - val_loss: 0.0062
Epoch 5/50
2/2 [==============================] - 2s 860ms/step - loss: 0.0041 - val_loss: 0.0051
Epoch 6/50
2/2 [==============================] - 2s 932ms/step - loss: 0.0035 - val_loss: 0.0043
Epoch 7/50
2/2 [==============================] - 2s 892ms/step - loss: 0.0028 - val_loss: 0.0039
Epoch 8/50
2/2 [==============================] - 2s 873ms/step - loss: 0.0024 - val_loss: 0.0036
Epoch 9/50
2/2 [==============================] - 2s 795ms/step - loss: 0.0021 - val_loss: 0.0035
Epoch 10/50
2/2 [==============================] - 2s 851ms/step - loss: 0.0018 - val_loss: 0.0032
Epoch 11/50
2/2 [========